<a href="https://colab.research.google.com/github/cuillerj/Webcam-monitor/blob/main/determineFoscamv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [381]:
import tensorflow as tf
import IPython.display as display
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
from pathlib import Path
import pandas as pd
import shutil
import random
import time
import glob
tf.__version__
from skimage import data
import matplotlib.pyplot as plt 
%matplotlib inline
from PIL import Image, ImageDraw, ImageFilter,ImageOps
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
print(tf.__version__)
import mysql.connector
domotique = mysql.connector.connect(
  host="192.168.1.199",
  user="jean",
  password="manu7890",
  database="domotiquedata"
)

print("sql connector:",domotique)

2.2.0
sql connector: <mysql.connector.connection.MySQLConnection object at 0x000001FF650B0EB0>


In [382]:
def insert_Picture(name,value,det,type):
   mycursor = domotique.cursor()
   sql = "INSERT INTO webCamPictures ( pictureTime, pictureName, pictureDeterminedValue, pictureDetermination,pictureType) VALUES (now(),%s,%s, %s,%s)"
   val = (name,value,det,type)
   mycursor.execute(sql, val)
   domotique.commit()

In [384]:
left = 15
top = 80
right = 1250
bottom = 720
left2 = 160
top2 = 280
right2 = 850
bottom2 = 600
labels =['ras','something']
IMG_WIDTH=round((right-left)/2)
IMG_HEIGHT=round((bottom-top)/2)
newsize =(IMG_WIDTH,IMG_HEIGHT)
IMG_WIDTH2=round((right2-left2)/2)
IMG_HEIGHT2=round((bottom2-top2)/2)
newsize2 =(IMG_WIDTH2,IMG_HEIGHT2)
imageQuality = 75
print(newsize2)
origin_dir='X:/foscam/'
checkpoint_path_Day = "E:/Colab/Foscam/trainingv3/cp.ckpt"
checkpoint_path_Night = "E:/Colab/Foscam/trainingNight/cp.ckpt"
day_Dir = pathlib.Path(os.path.join(origin_dir,'toDetermineDay2')) # must contain a sudirectory for test generator
#day_dir = pathlib.Path(os.path.join(origin_dir,'toDetermineDay2/files'))
night_Dir = pathlib.Path(os.path.join(origin_dir,'toDetermineNight'))
#night_dir = pathlib.Path(os.path.join(origin_dir,'toDetermineNight/files'))
print(day_dir)
print(night_dir)
running=True

(345, 160)
X:\foscam\toDetermineDay2\files
X:\foscam\toDetermineNight\files


In [385]:
modelDay = Sequential([
    Conv2D(8, 1, padding='same', activation='relu', input_shape=(IMG_HEIGHT2, IMG_WIDTH2 ,3)),
    MaxPooling2D(),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])
modelDay.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
modelDay.summary()
checkpoint_dir = os.path.dirname(checkpoint_path_Day)
latest = tf.train.latest_checkpoint(checkpoint_dir)
modelDay.load_weights(latest)
print("Day weights loaded")

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_115 (Conv2D)          (None, 160, 345, 8)       32        
_________________________________________________________________
max_pooling2d_115 (MaxPoolin (None, 80, 172, 8)        0         
_________________________________________________________________
conv2d_116 (Conv2D)          (None, 80, 172, 16)       1168      
_________________________________________________________________
max_pooling2d_116 (MaxPoolin (None, 40, 86, 16)        0         
_________________________________________________________________
conv2d_117 (Conv2D)          (None, 40, 86, 32)        4640      
_________________________________________________________________
max_pooling2d_117 (MaxPoolin (None, 20, 43, 32)        0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 20, 43, 64)      

In [386]:
modelNight= Sequential([
    Conv2D(8, 1, padding='same', activation='relu', input_shape=(IMG_HEIGHT2, IMG_WIDTH2 ,3)),
    MaxPooling2D(),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])
modelNight.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
modelNight.summary()
checkpoint_dir = os.path.dirname(checkpoint_path_Night)
latest = tf.train.latest_checkpoint(checkpoint_dir)
modelNight.load_weights(latest)
print("Night weights loaded")

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_120 (Conv2D)          (None, 160, 345, 8)       32        
_________________________________________________________________
max_pooling2d_120 (MaxPoolin (None, 80, 172, 8)        0         
_________________________________________________________________
conv2d_121 (Conv2D)          (None, 80, 172, 16)       1168      
_________________________________________________________________
max_pooling2d_121 (MaxPoolin (None, 40, 86, 16)        0         
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 40, 86, 32)        4640      
_________________________________________________________________
max_pooling2d_122 (MaxPoolin (None, 20, 43, 32)        0         
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 20, 43, 64)      

In [ ]:
while running:
  day_dir = pathlib.Path(os.path.join(origin_dir,'toDetermineDay2/files'))
  dayList = list(day_dir.glob('*.jpg'))
  filesNb=len(dayList)
  if (filesNb>0):
    print ("nombre de photos couleur à determiner:",filesNb)
    day_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our test data
    day_generator = day_image_generator.flow_from_directory(batch_size=filesNb,
                                                              directory=day_Dir,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT2, IMG_WIDTH2),
                                                              class_mode='binary')
    STEP_SIZE_TEST=day_generator.n //test_generator.batch_size
    day_generator.reset()
    predDay=modelDay.predict(day_generator)
    xDay=predDay>=0
    predicted_class_indices=[int(item) for item in xDay]
    predictions = [labels[k] for k in predicted_class_indices]
    values  = [predDay[k][0] for k in range(predDay.size)]
    x=max(predDay)
    idx=np.where(predDay==x)
    listArr=np.asarray(dayList)
    predicted_class_indices=[int(item) for item in xDay]
    predictions = [labels[k] for k in predicted_class_indices]
    values  = [predDay[k][0] for k in range(predDay.size)]
    print(x)
    id=idx[0]
    print(id)
    for i in range(0,(filesNb)):
      fname=str(listArr[i])
      idx=fname.find("MDAlarm")
      fname=fname[idx:]
      det=predictions[int(i)]
      value=float(predDay[int(i)])
      print(fname,det)
      insert_Picture(fname,value,det,1)
      if os.path.exists(str(listArr[i])):
        os.remove(str(listArr[i]))
      else:
        print(str(listArr[i])," does not exist")
      #  nightList = list(night_dir.glob('*.jpg'))
      if(len(nightList)>0):
        print ("nombre de photos N&B à determiner:",len(nightList))

  night_dir = pathlib.Path(os.path.join(origin_dir,'toDetermineNight/files'))
  nightList = list(night_dir.glob('*.jpg'))
  filesNb=len(nightList)
  if (filesNb>0):
    print ("nombre de photos N&B à determiner:",filesNb)
    night_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our test data
    night_generator = night_image_generator.flow_from_directory(batch_size=len(nightList),
                                                              directory=night_Dir,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT2, IMG_WIDTH2),
                                                              class_mode='binary')
 #   STEP_SIZE_TEST=day_generator.n //test_generator.batch_size
    night_generator.reset()
    predNight=modelNight.predict(night_generator)
    xNight=predNight>=0
    predicted_class_indices=[int(item) for item in xDay]
    predictions = [labels[k] for k in predicted_class_indices]
    values  = [predNight[k][0] for k in range(predNight.size)]
#    x=max(predDay)
 #   idx=np.where(predDay==x)
    listArr=np.asarray(nightList)
    predicted_class_indices=[int(item) for item in xNight]
    predictions = [labels[k] for k in predicted_class_indices]
    values  = [predNight[k][0] for k in range(predNight.size)]
 #   print(x)
  #  id=idx[0]
   # print(id)
    for i in range(0,(filesNb)):
      fname=str(listArr[i])
      idx=fname.find("MDAlarm")
      fname=fname[idx:]
      det=predictions[int(i)]
      value=float(predNight[int(i)])
      print(fname,det)
      insert_Picture(fname,value,det,2)
      if os.path.exists(str(listArr[i])):
        os.remove(str(listArr[i]))
      else:
        print(str(listArr[i])," does not exist")
      #  nightList = list(night_dir.glob('*.jpg'))
      if(len(nightList)>0):
        print ("nombre de photos N&B à determiner:",len(nightList))  
  time.sleep(30)

nombre de photos N&B à determiner: 87
Found 87 images belonging to 1 classes.
MDAlarm_20220315-194431.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194433.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194435.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194445.jpg something
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194447.jpg something
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194449.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194502.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194504.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194506.jpg something
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194535.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194537.jpg something
nombre de photos N&B à determiner: 87
MDAlarm_20220315-194539.jpg ras
nombre de photos N&B à determiner: 87
MDAlarm_20220315-195338.jpg ras
nombre de photos N&B à det

In [378]:
#day_dir = pathlib.Path(os.path.join(origin_dir,'toDetermineDay2/files'))
dayList = list(day_dir.glob('*.jpg'))
dayList

[WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-100948.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-101014.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-101016.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-101018.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-115943.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-115945.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-115947.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-115949.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-115951.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-115953.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-120000.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-120002.jpg'),
 WindowsPath('X:/foscam/toDetermineDay2/files/MDAlarm_20220315-120004.jpg'),

In [359]:
test_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our test data
day_generator = test_image_generator.flow_from_directory(batch_size=len(dayList),
                                                              directory=day_Dir,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT2, IMG_WIDTH2),
                                                              class_mode='binary')
STEP_SIZE_TEST=day_generator.n //test_generator.batch_size
test_generator.reset()
predDay=modelDay.predict(day_generator,steps=STEP_SIZE_TEST,verbose=1)
predDay.size

Found 3 images belonging to 1 classes.


AttributeError: ignored

In [341]:
xDay=predDay>=0
predicted_class_indices=[int(item) for item in xDay]
predictions = [labels[k] for k in predicted_class_indices]
values  = [predDay[k][0] for k in range(predDay.size)]
x=max(predDay)
idx=np.where(predDay==x)
listArr=np.asarray(dayList)
predicted_class_indices=[int(item) for item in xDay]
predictions = [labels[k] for k in predicted_class_indices]
values  = [predDay[k][0] for k in range(predDay.size)]
print(x)
id=idx[0]
print(id)
filesNb=len(dayList)
for i in range(0,filesNb-1):
  fname=str(listArr[i])
  idx=fname.find("MDAlarm")
  fname=fname[idx:]
  #fname=fname[:27]
  #print(fname)
  det=predictions[int(i)]
  value=float(predDay[int(i)])
  print(fname,det)
  insert_Picture(fname,value,det,1)
  if os.path.exists(str(listArr[i])):
    os.remove(str(listArr[i]))
  else:
    print(str(listArr[i])," does not exist")

[121.88619]
[3]
MDAlarm_20220315-084048.jpg something
MDAlarm_20220315-084050.jpg something
MDAlarm_20220315-084052.jpg something
MDAlarm_20220315-084107.jpg something
MDAlarm_20220315-084109.jpg something
MDAlarm_20220315-084111.jpg something


IndexError: ignored

In [313]:
night_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our test data
night_generator = test_image_generator.flow_from_directory(batch_size=len(nightList),
                                                              directory=night_Dir,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT2, IMG_WIDTH2),
                                                              class_mode='binary')
STEP_SIZE_TEST=night_generator.n //test_generator.batch_size
test_generator.reset()
predNight=modelNight.predict(night_generator,steps=STEP_SIZE_TEST,verbose=1)
predNight.size

Found 24 images belonging to 1 classes.
1/4 [======>.......................] - 0s 9ms/step


24

In [315]:
xNight=predNight>=0
predicted_class_indices=[int(item) for item in xNight]
predictions = [labels[k] for k in predicted_class_indices]
values  = [predNight[k][0] for k in range(predNight.size)]
x=max(predNight)
idx=np.where(predNight==x)
listArr=np.asarray(nightList)
predicted_class_indices=[int(item) for item in xNight]
predictions = [labels[k] for k in predicted_class_indices]
values  = [predDay[k][0] for k in range(predDay.size)]
print(x)
id=idx[0]
print(id)
for i in range(0,len(filenames)):
  fname=str(listArr[i])
  idx=fname.find("MDAlarm")
  fname=fname[idx:]
  #fname=fname[:27]
  #print(fname)
  det=predictions[int(i)]
  value=float(predNight[int(i)])
  print(fname,det)
  insert_Picture(fname,value,det,2)
  if os.path.exists(str(listArr[i])):
    os.remove(str(listArr[i]))
  else:
    print(str(listArr[i])," does not exist")

[-7.4217753]
[21]
MDAlarm_20220314-210236.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220314-210236.jpg  does not exist
MDAlarm_20220314-210238.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220314-210238.jpg  does not exist
MDAlarm_20220314-210240.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220314-210240.jpg  does not exist
MDAlarm_20220314-215833.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220314-215833.jpg  does not exist
MDAlarm_20220314-215835.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220314-215835.jpg  does not exist
MDAlarm_20220314-215837.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220314-215837.jpg  does not exist
MDAlarm_20220315-034106.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220315-034106.jpg  does not exist
MDAlarm_20220315-034108.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220315-034108.jpg  does not exist
MDAlarm_20220315-034110.jpg ras
X:\foscam\toDetermineNight\files\MDAlarm_20220315-034110.jpg  does not exist
M

IndexError: ignored

In [273]:
domotique.commit()